In [7]:
from torch.nn import Module, Conv2d, Linear, MaxPool2d, ReLU, Softmax, Flatten, Dropout, BCELoss
from torch.optim import Adam
import torch

In [8]:
class Unet(Module):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv2d(480000, 64, (5, 5), stride=(1,1), padding_mode="zeros")
        self.relu1 = ReLU()
        self.conv2 = Conv2d(64, 64, (5, 5), stride=(1,1), padding_mode="zeros")
        self.relu2 = ReLU()
        self.maxpool1 = MaxPool2d(kernel_size=(2, 2), stride=(1, 1))
        self.conv3 = Conv2d(64, 128, (5, 5), stride=(1,1), padding_mode="zeros")
        self.relu3 = ReLU()
        self.conv4 = Conv2d(128, 128, (5, 5), stride=(1,1), padding_mode="zeros")
        self.relu4 = ReLU()
        self.maxpool2 = MaxPool2d(kernel_size=(2, 2), stride=(1, 1))
        self.flatten = Flatten()
        self.fc1 = Linear(16384, 128)
        self.dropout1 = Dropout(0.25)
        self.fc2 = Linear(128, 64)
        self.dropout2 = Dropout(0.25)
        self.output = Softmax(2)
    def forward(self, x):
        x = self.conv1(x)
        x  = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool1(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.conv4(x)
        x = self.relu4(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.droput1(x)
        x = self.fc2(x)
        x = self.dropout2(x)
        x = self.output(x)
        return x
        
        


In [ ]:
from torch.utils.data import Dataset, DataLoader
import os

class ImageDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.images = df[df['ty']]
        print("initialized")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx].replace(".jpg", "_mask.png"))
        
        matting, color = load_img(self.df, )
        
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)
        
        return image, mask


train_dataset = ImageDataset()
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)


In [ ]:
model = Unet()
loss = BCELoss()
opt = Adam(model.parameters(), lr=0.001)
epochs = 20

for epoch in range(epochs):
    model.train()
    for images, masks in train_loader:
        y_pred = model(images)
        current_loss = loss(y_pred, masks)
        opt.zero_grad()
        loss.backward()
        opt.step()

    print("Epoch" + str(epoch) + " Loss: " + str(loss.item()))
        